In [2]:
import pandas as pd
import sympy as sp
import statsmodels.api as sm
import pyomo.environ as pyo

In [118]:
class residentialModule:
    prices = {}
    loads = {}
    hr_to_s = pd.DataFrame()
    baseYear = 2023

    def __init__(self):
        self.year = sp.Idx('year')
        self.reg = sp.Idx('region')
        self.fuel = sp.Idx('fuel')
        self.LastHYr, self.LastMYr, self.BaseYr = sp.symbols(('LastHYr','LastMYr','base'))

        self.income = sp.IndexedBase('Income')
        self.incomeIndex = sp.IndexedBase('IncomeIndex')
        self.i_elas = sp.IndexedBase('IncomeElasticity')
        self.i_lag = sp.IndexedBase('IncomeLagParameter')

        self.price = sp.IndexedBase('Price')
        self.priceIndex = sp.IndexedBase('PriceIndex')
        self.p_elas = sp.IndexedBase('PriceElasticity')
        self.p_lag = sp.IndexedBase('PriceLagParameter')
        
        self.trendGR = sp.IndexedBase('TrendGR')

        self.consumption = sp.IndexedBase('Consumption')

        self.incomeEQ = (self.incomeIndex[self.year-1,self.reg,self.fuel] ** self.i_lag[self.reg,self.fuel]) * (self.income[self.year,self.reg,self.fuel]/self.income[self.BaseYr,self.reg,self.fuel]) ** self.i_elas[self.reg,self.fuel]
        self.priceEQ = (self.priceIndex[self.year-1,self.reg,self.fuel] ** self.p_lag[self.reg,self.fuel]) * (self.price[self.year,self.reg,self.fuel]/self.price[self.BaseYr,self.reg,self.fuel]) ** self.p_elas[self.reg,self.fuel]
        self.growthEQ = 1 + ((self.year - self.LastHYr)/(self.LastMYr - self.LastHYr)) * (((1 + self.trendGR[self.reg,self.fuel]) ** (self.LastMYr - self.LastHYr)) - 1)

        self.QIndex = self.incomeEQ * self.priceEQ * self.growthEQ

        self.demand = self.consumption[self.BaseYr,self.reg,self.fuel] * self.QIndex

        self.lambdifiedDemand = sp.lambdify([self.incomeIndex[self.year-1,self.reg,self.fuel],
                                    self.i_lag[self.reg,self.fuel],
                                    self.income[self.year,self.reg,self.fuel],
                                    self.income[self.BaseYr,self.reg,self.fuel],
                                    self.i_elas[self.reg,self.fuel],
                                    self.priceIndex[self.year-1,self.reg,self.fuel],
                                    self.p_lag[self.reg,self.fuel],
                                    self.price[self.year,self.reg,self.fuel],
                                    self.price[self.BaseYr,self.reg,self.fuel],
                                    self.p_elas[self.reg,self.fuel],
                                    self.year,
                                    self.LastHYr,
                                    self.LastMYr,
                                    self.trendGR[self.reg,self.fuel],
                                    self.consumption[self.BaseYr,self.reg,self.fuel]],
                                    self.demand)
        
        if not self.prices:
            priceData = pd.read_excel('../input/cem_elec_prices.xlsx').set_index(['r','y','hr'],drop=False)
            temploadData = pd.read_csv('../input/cem_inputs/Load.csv')
            temploadData = temploadData.loc[temploadData.y == 2023].set_index(['y','hr'],drop=False)
            cols = {f'r{i}':i for i in range(1,26)}
            temploadData = temploadData.rename(columns=cols)
            loadData = temploadData.loc[:,range(1,26)].stack().reset_index().rename(columns={'level_2':'r',0:'Load'}).set_index(['r','y','hr'],drop=False)
            # loadData = pd.DataFrame([(r,2023,hr) for r in range(1,26) for hr in range(1,8761)],columns=['r','y','hr']).set_index(['r','y','hr'],drop=False)
            # loadData['Load'] = loadData.apply(lambda row: temploadData.loc[(2023,row.hr),str(row.r)],axis=1)
            self.set_base_values(priceData,loadData)
        
        self.demandF = lambda price, load, year, basePrice = 1, p_elas = -0.10, baseYear = self.baseYear, baseIncome = 1, income = 1, i_elas = 1, priceIndex = 1, incomeIndex = 1, p_lag = 1, i_lag = 1, trend = 0 : \
            self.lambdifiedDemand(incomeIndex, i_lag, income, baseIncome, i_elas, priceIndex, p_lag, price, basePrice, p_elas, year, baseYear, 2050, trend, load)
        
        pass
    
    #Sets up base values
    def set_base_values(self, p, load):
        self.prices['BasePrices'] = p
        self.baseYear = p.y.unique()
        self.loads['BaseLoads'] = load.Load
        return
    
    def update_load(self, p):
        hours = p.hr.unique()
        n = len(hours)
        newLoad = p.copy()
        hourMap = {}
        #hard-coded for the 4 seasons from 8760 data
        hourMap[1] = list(range(1,2161)) + list(range(8017,8761))
        hourMap[3] = range(2161,3625)
        hourMap[2] = range(3625,6553)
        hourMap[4] = range(6553,8017)
        # for h in hours:
        #     hourMap[h] = range(int(((h-1)*8760/n)+1),int((h*8760/n)+1))
        newLoad['BasePrice'] = newLoad.apply(lambda row: sum(self.prices['BasePrices'].loc[(row.r,2023,hr),'Dual'] for hr in hourMap[row.hr])/(len(hourMap[row.hr])),axis=1)
        newLoad['Load'] = newLoad.apply(lambda row: self.demandF(row.Dual,self.loads['BaseLoads'].loc[(row.r,2023,row.hr)],row.y,row.BasePrice)[0],axis=1)
        return newLoad.Load
    
    #Creates a block that can be given to another pyomo model
    #The constraint is essentially just updating the load parameter
    def make_block(self, prices):
        loadIndex = []
        for i in prices.index:
            loadIndex.append((i[0],2023,i[2]))
        mod = pyo.ConcreteModel()
        mod.block = pyo.Block()

        mod.block.price_set = pyo.Set(initialize=prices.index)
        mod.block.load_set = pyo.Set(initialize=loadIndex)

        mod.block.prices = pyo.Param(mod.block.price_set, initialize=prices.Dual, mutable=True)
        mod.block.base_load = pyo.Param(mod.block.load_set, initialize=self.loads['BaseLoads'].loc[loadIndex], mutable=True)
        updated_load = self.update_load(prices)

        mod.block.Load = pyo.Var(mod.block.price_set, within=pyo.NonNegativeReals)

        @mod.block.Constraint(mod.block.price_set)
        def create_load(block,r,y,hr):
            return block.Load[r,y,hr] == updated_load.loc[(r,y,hr)]
        
        return mod.block
        
    #This is a standalone pyomo model
    #The objective function doesn't have much meaning
    #The real purpose is to have the load parameter changed into a variable that gets updated by the new prices
    def make_pyomo_model(self, prices):
        loadIndex = []
        for i in prices.index:
            loadIndex.append((i[0],2023,i[2]))
        mod = pyo.ConcreteModel()

        mod.price_set = pyo.Set(initialize=prices.index)
        mod.load_set = pyo.Set(initialize=loadIndex)
        
        mod.prices = pyo.Param(mod.price_set, initialize=prices.Dual, mutable=True)
        mod.base_load = pyo.Param(mod.load_set, initialize=self.loads['BaseLoads'].loc[loadIndex], mutable=True)
        updated_load = self.update_load(prices)
        
        mod.Load = pyo.Var(mod.price_set, within=pyo.NonNegativeReals)

        mod.obj = pyo.Objective(rule = sum(mod.Load[r,y,hr] for [r,y,hr] in mod.price_set))

        @mod.Constraint(mod.price_set)
        def create_new_load(mod,r,y,hr):
            return mod.Load[r,y,hr] == updated_load.loc[(r,y,hr)]
        
        return mod

In [119]:
model = residentialModule()

In [ ]:
newPrices = pd.DataFrame([(1,2024,1,11.59),(1,2024,2,12.25),(1,2024,3,14.81),(1,2024,4,11.53)],columns=['r','y','hr','Dual']).set_index(['r','y','hr'],drop=False)

In [120]:
model2 = residentialModule()

In [121]:
newBlock = model.make_block(newPrices)

In [122]:
newBlock.pprint()

2 Set Declarations
    load_set : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     3 :    Any :    4 : {(1, 2023, 1), (1, 2023, 2), (1, 2023, 3), (1, 2023, 4)}
    price_set : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     3 :    Any :    4 : {(1, 2024, 1), (1, 2024, 2), (1, 2024, 3), (1, 2024, 4)}

2 Param Declarations
    base_load : Size=4, Index=load_set, Domain=Any, Default=None, Mutable=True
        Key          : Value
        (1, 2023, 1) :  47.8
        (1, 2023, 2) :  46.6
        (1, 2023, 3) :  45.8
        (1, 2023, 4) :  45.5
    prices : Size=4, Index=price_set, Domain=Any, Default=None, Mutable=True
        Key          : Value
        (1, 2024, 1) : 11.59
        (1, 2024, 2) : 12.25
        (1, 2024, 3) : 14.81
        (1, 2024, 4) : 11.53

1 Var Declarations
    Load : Size=4, Index=price_set
        Key          : Lower : Value : Upper : Fixed : Stale : Do

In [81]:
newModel = model.make_pyomo_model(newPrices)
newModel.pprint()

2 Set Declarations
    load_set : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     3 :    Any :    4 : {(1, 2023, 1), (1, 2023, 2), (1, 2023, 3), (1, 2023, 4)}
    price_set : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     3 :    Any :    4 : {(1, 2024, 1), (1, 2024, 2), (1, 2024, 3), (1, 2024, 4)}

2 Param Declarations
    base_load : Size=4, Index=load_set, Domain=Any, Default=None, Mutable=True
        Key          : Value
        (1, 2023, 1) :  47.8
        (1, 2023, 2) :  46.6
        (1, 2023, 3) :  45.8
        (1, 2023, 4) :  45.5
    prices : Size=4, Index=price_set, Domain=Any, Default=None, Mutable=True
        Key          : Value
        (1, 2024, 1) : 11.59
        (1, 2024, 2) : 12.25
        (1, 2024, 3) : 14.81
        (1, 2024, 4) : 11.53

1 Var Declarations
    Load : Size=4, Index=price_set
        Key          : Lower : Value : Upper : Fixed : Stale : Do

In [112]:
temploadData = pd.read_csv('../input/cem_inputs/Load.csv')
temploadData = temploadData.loc[temploadData.y == 2023].set_index(['y','hr'],drop=False)
loadData = pd.DataFrame([(r,2023,hr) for r in range(1,26) for hr in range(1,8761)],columns=['r','y','hr']).set_index(['r','y','hr'],drop=False)
loadData['Load'] = loadData.apply(lambda row: temploadData.loc[(2023,row.hr),str(row.r)],axis=1)

cols = {str(i):i for i in range(1,26)}
temploadData = temploadData.rename(columns=cols)
test = temploadData.loc[:,range(1,26)].stack().reset_index().rename(columns={'level_2':'r',0:'Load'}).set_index(['r','y','hr'],drop=False)


In [111]:
test.r.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25'], dtype=object)